In [ ]:
from tkinter import *
import tkinter.messagebox as box
import tkinter as tk 
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

win = Tk()
win.geometry("600x100")
win.title("데이터 창")

# 얼굴 인식용 haar/cascade 로딩
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')    
def Certified():
    # 사용자 얼굴 학습
    def train(name):
        data_path = 'faces/' + name + '/'
        #파일만 리스트로 만듬
        face_pics = [f for f in listdir(data_path) if isfile(join(data_path,f))]

        Training_Data, Labels = [], []

        for i, files in enumerate(face_pics):
            image_path = data_path + face_pics[i]
            images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            # 이미지가 아니면 패스
            if images is None:
                continue    
            Training_Data.append(np.asarray(images, dtype=np.uint8))
            Labels.append(i)
        if len(Labels) == 0:
            print("There is no data to train.")
            return None
        Labels = np.asarray(Labels, dtype=np.int32)
        # 모델 생성
        model = cv2.face.LBPHFaceRecognizer_create()
        # 학습
        model.train(np.asarray(Training_Data), np.asarray(Labels))
        print(name + " : Model Training Complete!!!!!")

        #학습 모델 리턴
        return model

    # 여러 사용자 학습
    def trains():
        #faces 폴더의 하위 폴더를 학습
        data_path = 'faces/'
        # 폴더만 색출
        model_dirs = [f for f in listdir(data_path) if listdir(join(data_path,f))]

        #학습 모델 저장할 딕셔너리
        models = {}
        # 각 폴더에 있는 얼굴들 학습
        for model in model_dirs:
            print('model :' + model)
            # 학습 시작
            result = train(model)
            # 학습이 안되었다면 패스!
            if result is None:
                continue
            # 학습되었으면 저장
            print('model2 :' + model)
            models[model] = result

        # 학습된 모델 딕셔너리 리턴
        return models    

    #얼굴 검출
    def face_detector(img, size = 0.5):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray,1.3,5)
            if faces is():
                return img,[]
            for(x,y,w,h) in faces:
                cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,255),2)
                roi = img[y:y+h, x:x+w]
                roi = cv2.resize(roi, (200,200))
            return img,roi   #검출된 좌표에 사각 박스 그리고(img), 검출된 부위를 잘라(roi) 전달

    # 인식 시작
    def run(models):    
        #카메라 열기 
        cap = cv2.VideoCapture(0)

        while True:
            #카메라로 부터 사진 한장 읽기 
            ret, frame = cap.read()
            # 얼굴 검출 시도 
            image, face = face_detector(frame)
            try:            
                min_score = 999       #가장 낮은 점수로 예측된 사람의 점수
                min_score_name = ""   #가장 높은 점수로 예측된 사람의 이름

                #검출된 사진을 흑백으로 변환 
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

                #위에서 학습한 모델로 예측시도
                for key, model in models.items():
                    result = model.predict(face)                
                    if min_score > result[1]:
                        min_score = result[1]
                        min_score_name = key

                #min_score 신뢰도이고 0에 가까울수록 자신과 같다는 뜻이다.         
                if min_score < 500:
                    #????? 어쨋든 0~100표시하려고 한듯 
                    confidence = int(100*(1-(min_score)/300))
                    # 유사도 화면에 표시 
                    display_string = str(confidence)+'% Confidence it is ' + min_score_name
                cv2.putText(image,display_string,(100,120), cv2.FONT_HERSHEY_COMPLEX,1,(250,120,255),2)
                #85 보다 크면 동일 인물로 간주해 UnLocked! 
                if confidence > 85:
                    cv2.putText(image, "Unlocked : " + min_score_name, (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                    cv2.imshow('Face Cropper', image)
                    box.showinfo( '얼굴인증' , id+'님의 얼굴이 인증되었습니다 ' ) 
                    cv2.destroyAllWindows()
                    break
                else:
                #75 이하면 타인.. Locked!!! 
                    cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                    cv2.imshow('Face Cropper', image)
            except:
                #얼굴 검출 안됨 
                cv2.putText(image, "Face Not Found", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
                cv2.imshow('Face Cropper', image)
                pass
            if cv2.waitKey(1)==13:
                break
        cap.release()
        cv2.destroyAllWindows()

    if __name__ == "__main__":
        # 학습 시작
        models = trains()
        # 고!
        run(models)
btn0 = Button(win, text='데이터 올리기', width=25,command=Certified )
btn1 = Button(win, text='데이터 다운받기', width=25 , command=Certified )
btn0.pack() 
btn1.pack() 
win.mainloop()   